In [13]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd

import datetime as dt
import time
from zoneinfo import ZoneInfo

import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

from IPython.display import display, Markdown


In [2]:
import utils

In [126]:
import altair as alt

In [106]:
pd.set_option("display.max_columns", 100)


In [3]:
itp_id = 300
analysis_date = '2022-05-07'

In [23]:
gtfs_daily = (
        tbl.views.gtfs_schedule_fact_daily_trips()
        >> filter(_.calitp_itp_id == itp_id)
        >> filter(_.service_date == analysis_date)
        >> filter(_.is_in_service == True)
        >> collect()
    )

In [24]:
rt = query_sql(
        f"""
        SELECT * FROM `cal-itp-data-infra-staging.natalie_views.test_rt_trips`
        WHERE date = '{analysis_date}'
        """
    )

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [25]:
gtfs_daily.head()

,feed_key,trip_key,trip_id,route_id,calitp_itp_id,calitp_url_number,service_id,service_date,service_indicator,service_start_date,...,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours
0,4100255323842349957,4736682379560348113,880909,3480,300,0,1,2022-05-07,1,2022-03-27,...,None,None,True,2022-03-29,2022-07-18,28,28,56400,58680,0.633333
1,4100255323842349957,4656797115770312173,882712,3490,300,0,1,2022-05-07,1,2022-03-27,...,None,None,True,2022-03-29,2022-07-18,34,34,36300,38280,0.550000
2,4100255323842349957,5891417018716807450,882994,3493,300,0,1,2022-05-07,1,2022-03-27,...,None,None,True,2022-03-29,2022-07-18,35,36,24600,26700,0.583333
3,4100255323842349957,5368523724823864936,882998,3493,300,0,1,2022-05-07,1,2022-03-27,...,None,None,True,2022-03-29,2022-07-18,35,35,63960,66540,0.716667
4,4100255323842349957,8281954761629192567,881239,3481,300,0,1,2022-05-07,1,2022-03-27,...,None,None,True,2022-03-29,2022-07-18,39,39,32400,35580,0.883333


In [81]:
gtfs_daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 870 entries, 0 to 869
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   feed_key                 870 non-null    int64  
 1   trip_key                 870 non-null    int64  
 2   trip_id                  870 non-null    object 
 3   route_id                 870 non-null    object 
 4   calitp_itp_id            870 non-null    int64  
 5   calitp_url_number        870 non-null    int64  
 6   service_id               870 non-null    object 
 7   service_date             870 non-null    object 
 8   service_indicator        870 non-null    object 
 9   service_start_date       870 non-null    object 
 10  service_end_date         870 non-null    object 
 11  service_inclusion        0 non-null      object 
 12  service_exclusion        0 non-null      object 
 13  is_in_service            870 non-null    bool   
 14  calitp_extracted_at      8

In [26]:
rt.head()

,calitp_itp_id,calitp_url_number,date,trip_id,vehicle_id
0,300,0,2022-05-07,882099,1301
1,300,0,2022-05-07,882132,1301
2,300,0,2022-05-07,882158,1301
3,300,0,2022-05-07,882114,1301
4,300,0,2022-05-07,882162,1301


In [65]:
#right: rt 
#left: scheudle 
join = (pd.merge(gtfs_daily, rt, how='outer', on='trip_id', indicator='have_rt'))


In [28]:
join.have_rt.value_counts()

both          608
right_only    395
left_only     266
Name: have_rt, dtype: int64

In [86]:
join["first_departure"] = pd.to_datetime(join.trip_first_departure_ts, unit='s').dt.time


In [87]:
join

,feed_key,trip_key,trip_id,route_id,calitp_itp_id_x,calitp_url_number_x,service_id,service_date,service_indicator,service_start_date,...,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours,calitp_itp_id_y,calitp_url_number_y,date,vehicle_id,have_rt,first_departure
0,4.100255e+18,4.736682e+18,880909,3480,300.0,0.0,1,2022-05-07,1,2022-03-27,...,28.0,56400.0,58680.0,0.633333,300.0,0.0,2022-05-07,2111,both,15:40:00
1,4.100255e+18,4.656797e+18,882712,3490,300.0,0.0,1,2022-05-07,1,2022-03-27,...,34.0,36300.0,38280.0,0.550000,300.0,0.0,2022-05-07,1316,both,10:05:00
2,4.100255e+18,5.891417e+18,882994,3493,300.0,0.0,1,2022-05-07,1,2022-03-27,...,36.0,24600.0,26700.0,0.583333,300.0,0.0,2022-05-07,1602,both,06:50:00
3,4.100255e+18,5.368524e+18,882998,3493,300.0,0.0,1,2022-05-07,1,2022-03-27,...,35.0,63960.0,66540.0,0.716667,NaN,NaN,NaN,NaN,left_only,17:46:00
4,4.100255e+18,8.281955e+18,881239,3481,300.0,0.0,1,2022-05-07,1,2022-03-27,...,39.0,32400.0,35580.0,0.883333,NaN,NaN,NaN,NaN,left_only,09:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1264,NaN,NaN,881433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,5317,right_only,NaT
1265,NaN,NaN,881522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,5317,right_only,NaT
1266,NaN,NaN,881442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,5317,right_only,NaT
1267,NaN,NaN,881499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,5317,right_only,NaT


In [88]:
#rt leftover
rt_lo =(join>>filter(_.have_rt=='right_only'))

In [89]:
rt_lo

,feed_key,trip_key,trip_id,route_id,calitp_itp_id_x,calitp_url_number_x,service_id,service_date,service_indicator,service_start_date,...,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours,calitp_itp_id_y,calitp_url_number_y,date,vehicle_id,have_rt,first_departure
874,NaN,NaN,882099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,1301,right_only,NaT
875,NaN,NaN,882132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,1301,right_only,NaT
876,NaN,NaN,882158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,1301,right_only,NaT
877,NaN,NaN,882114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,1301,right_only,NaT
878,NaN,NaN,882162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,1301,right_only,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1264,NaN,NaN,881433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,5317,right_only,NaT
1265,NaN,NaN,881522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,5317,right_only,NaT
1266,NaN,NaN,881442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,5317,right_only,NaT
1267,NaN,NaN,881499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,5317,right_only,NaT


In [90]:
#schedule
sched_lo = (join>>filter(_.have_rt=='left_only'))

In [91]:
sched_lo

,feed_key,trip_key,trip_id,route_id,calitp_itp_id_x,calitp_url_number_x,service_id,service_date,service_indicator,service_start_date,...,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours,calitp_itp_id_y,calitp_url_number_y,date,vehicle_id,have_rt,first_departure
3,4.100255e+18,5.368524e+18,882998,3493,300.0,0.0,1,2022-05-07,1,2022-03-27,...,35.0,63960.0,66540.0,0.716667,NaN,NaN,NaN,NaN,left_only,17:46:00
4,4.100255e+18,8.281955e+18,881239,3481,300.0,0.0,1,2022-05-07,1,2022-03-27,...,39.0,32400.0,35580.0,0.883333,NaN,NaN,NaN,NaN,left_only,09:00:00
6,4.100255e+18,-6.728747e+18,881204,3481,300.0,0.0,1,2022-05-07,1,2022-03-27,...,39.0,63720.0,67740.0,1.116667,NaN,NaN,NaN,NaN,left_only,17:42:00
7,4.100255e+18,2.093077e+18,880502,3479,300.0,0.0,1,2022-05-07,1,2022-03-27,...,42.0,37920.0,40800.0,0.800000,NaN,NaN,NaN,NaN,left_only,10:32:00
8,4.100255e+18,-9.117673e+17,880589,3479,300.0,0.0,1,2022-05-07,1,2022-03-27,...,45.0,72180.0,75000.0,0.783333,NaN,NaN,NaN,NaN,left_only,20:03:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,4.100255e+18,3.630855e+18,880533,3479,300.0,0.0,1,2022-05-07,1,2022-03-27,...,45.0,81840.0,83880.0,0.566667,NaN,NaN,NaN,NaN,left_only,22:44:00
866,4.100255e+18,-6.262297e+18,880583,3479,300.0,0.0,1,2022-05-07,1,2022-03-27,...,45.0,64560.0,67980.0,0.950000,NaN,NaN,NaN,NaN,left_only,17:56:00
867,4.100255e+18,6.841902e+18,880584,3479,300.0,0.0,1,2022-05-07,1,2022-03-27,...,45.0,66540.0,69660.0,0.866667,NaN,NaN,NaN,NaN,left_only,18:29:00
870,4.100255e+18,-1.020052e+18,882000,3486,300.0,0.0,1,2022-05-07,1,2022-03-27,...,48.0,65640.0,68820.0,0.883333,NaN,NaN,NaN,NaN,left_only,18:14:00


In [92]:
rt_stime = query_sql(
        f"""
        SELECT * FROM `cal-itp-data-infra-staging.natalie_views.test_rt_trips_05072022`
        WHERE date = '{analysis_date}'
        """
    )

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [94]:
rt_stime["start_time"] = pd.to_datetime(rt_stime.start_time, unit='s').dt.time


In [95]:
rt_stime

,calitp_itp_id,calitp_url_number,date,trip_id,start_time
0,300,0,2022-05-07,883191,15:40:48
1,300,0,2022-05-07,882394,17:14:40
2,300,0,2022-05-07,881160,18:57:04
3,300,0,2022-05-07,880812,04:05:20
4,300,0,2022-05-07,882407,22:51:44
...,...,...,...,...,...
994,300,0,2022-05-07,882333,14:15:27
995,300,0,2022-05-07,881174,15:32:15
996,300,0,2022-05-07,882316,02:27:11
997,300,0,2022-05-07,881648,20:52:15


In [99]:
#joining the leftover realtime and join

test_sched = (pd.merge(sched_lo, rt_stime, on='trip_id', how='outer', indicator='_merge'))

In [100]:
test_sched

,feed_key,trip_key,trip_id,route_id,calitp_itp_id_x,calitp_url_number_x,service_id,service_date,service_indicator,service_start_date,...,calitp_url_number_y,date_x,vehicle_id,have_rt,first_departure,calitp_itp_id,calitp_url_number,date_y,start_time,_merge
0,4.100255e+18,5.368524e+18,882998,3493,300.0,0.0,1,2022-05-07,1,2022-03-27,...,NaN,NaN,NaN,left_only,17:46:00,NaN,NaN,NaN,NaN,left_only
1,4.100255e+18,8.281955e+18,881239,3481,300.0,0.0,1,2022-05-07,1,2022-03-27,...,NaN,NaN,NaN,left_only,09:00:00,NaN,NaN,NaN,NaN,left_only
2,4.100255e+18,-6.728747e+18,881204,3481,300.0,0.0,1,2022-05-07,1,2022-03-27,...,NaN,NaN,NaN,left_only,17:42:00,NaN,NaN,NaN,NaN,left_only
3,4.100255e+18,2.093077e+18,880502,3479,300.0,0.0,1,2022-05-07,1,2022-03-27,...,NaN,NaN,NaN,left_only,10:32:00,NaN,NaN,NaN,NaN,left_only
4,4.100255e+18,-9.117673e+17,880589,3479,300.0,0.0,1,2022-05-07,1,2022-03-27,...,NaN,NaN,NaN,left_only,20:03:00,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1260,NaN,NaN,882333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,14:15:27,right_only
1261,NaN,NaN,881174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,15:32:15,right_only
1262,NaN,NaN,882316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,02:27:11,right_only
1263,NaN,NaN,881648,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,20:52:15,right_only


In [101]:
test_sched._merge.value_counts()

right_only    999
left_only     266
both            0
Name: _merge, dtype: int64

In [102]:
#leftover schedule
test_rt = (pd.merge(rt_lo, rt_stime, on='trip_id', how='outer', indicator='_merge'))

In [103]:
test_rt

,feed_key,trip_key,trip_id,route_id,calitp_itp_id_x,calitp_url_number_x,service_id,service_date,service_indicator,service_start_date,...,calitp_url_number_y,date_x,vehicle_id,have_rt,first_departure,calitp_itp_id,calitp_url_number,date_y,start_time,_merge
0,NaN,NaN,882099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,2022-05-07,1301,right_only,NaT,300,0,2022-05-07,23:58:11,both
1,NaN,NaN,882132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,2022-05-07,1301,right_only,NaT,300,0,2022-05-07,03:00:05,both
2,NaN,NaN,882158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,2022-05-07,1301,right_only,NaT,300,0,2022-05-07,04:47:44,both
3,NaN,NaN,882114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,2022-05-07,1301,right_only,NaT,300,0,2022-05-07,02:17:06,both
4,NaN,NaN,882162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,2022-05-07,1301,right_only,NaT,300,0,2022-05-07,00:41:44,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,NaN,NaN,881604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,300,0,2022-05-07,13:32:46,right_only
995,NaN,NaN,882333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,300,0,2022-05-07,14:15:27,right_only
996,NaN,NaN,881174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,300,0,2022-05-07,15:32:15,right_only
997,NaN,NaN,881648,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,300,0,2022-05-07,20:52:15,right_only


In [104]:
test_rt._merge.value_counts()

right_only    604
both          395
left_only       0
Name: _merge, dtype: int64

In [108]:
test_rt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 999 entries, 0 to 998
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   feed_key                 0 non-null      float64 
 1   trip_key                 0 non-null      float64 
 2   trip_id                  999 non-null    object  
 3   route_id                 0 non-null      object  
 4   calitp_itp_id_x          0 non-null      float64 
 5   calitp_url_number_x      0 non-null      float64 
 6   service_id               0 non-null      object  
 7   service_date             0 non-null      object  
 8   service_indicator        0 non-null      object  
 9   service_start_date       0 non-null      object  
 10  service_end_date         0 non-null      object  
 11  service_inclusion        0 non-null      object  
 12  service_exclusion        0 non-null      object  
 13  is_in_service            0 non-null      object  
 14  calitp_ext

In [113]:
(test_rt>>filter(_._merge=='both')
 >>select(_.trip_id,
          _.calitp_itp_id_y,
          _.calitp_url_number_y,
          _.have_rt,
          _.calitp_itp_id,
          _.calitp_url_number,
          _.date_y,
          _.start_time,
          _._merge)
)

,trip_id,calitp_itp_id_y,calitp_url_number_y,have_rt,calitp_itp_id,calitp_url_number,date_y,start_time,_merge
0,882099,300.0,0.0,right_only,300,0,2022-05-07,23:58:11,both
1,882132,300.0,0.0,right_only,300,0,2022-05-07,03:00:05,both
2,882158,300.0,0.0,right_only,300,0,2022-05-07,04:47:44,both
3,882114,300.0,0.0,right_only,300,0,2022-05-07,02:17:06,both
4,882162,300.0,0.0,right_only,300,0,2022-05-07,00:41:44,both
...,...,...,...,...,...,...,...,...,...
390,881433,300.0,0.0,right_only,300,0,2022-05-07,01:06:00,both
391,881522,300.0,0.0,right_only,300,0,2022-05-07,05:52:31,both
392,881442,300.0,0.0,right_only,300,0,2022-05-07,03:10:10,both
393,881499,300.0,0.0,right_only,300,0,2022-05-07,23:58:13,both


In [125]:
test_sched

,feed_key,trip_key,trip_id,route_id,calitp_itp_id_x,calitp_url_number_x,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours,calitp_itp_id_y,calitp_url_number_y,date_x,vehicle_id,have_rt,first_departure,calitp_itp_id,calitp_url_number,date_y,start_time,_merge
0,4.100255e+18,5.368524e+18,882998,3493,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,35.0,35.0,63960.0,66540.0,0.716667,NaN,NaN,NaN,NaN,left_only,17:46:00,NaN,NaN,NaN,NaN,left_only
1,4.100255e+18,8.281955e+18,881239,3481,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,39.0,39.0,32400.0,35580.0,0.883333,NaN,NaN,NaN,NaN,left_only,09:00:00,NaN,NaN,NaN,NaN,left_only
2,4.100255e+18,-6.728747e+18,881204,3481,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,39.0,39.0,63720.0,67740.0,1.116667,NaN,NaN,NaN,NaN,left_only,17:42:00,NaN,NaN,NaN,NaN,left_only
3,4.100255e+18,2.093077e+18,880502,3479,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,42.0,42.0,37920.0,40800.0,0.800000,NaN,NaN,NaN,NaN,left_only,10:32:00,NaN,NaN,NaN,NaN,left_only
4,4.100255e+18,-9.117673e+17,880589,3479,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,45.0,45.0,72180.0,75000.0,0.783333,NaN,NaN,NaN,NaN,left_only,20:03:00,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1260,NaN,NaN,882333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,14:15:27,right_only
1261,NaN,NaN,881174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,15:32:15,right_only
1262,NaN,NaN,882316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,02:27:11,right_only
1263,NaN,NaN,881648,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,20:52:15,right_only


In [124]:
(test_sched>>filter(_._merge=='left_only')
 >>select(_.trip_id,
          _.route_id,
          _.route_id,
          _.first_departure,
          _.calitp_itp_id_x,
          _.calitp_url_number_x,
          _.have_rt,
          _.calitp_itp_id,
          _.calitp_url_number,
          _.date_x,
          _.start_time,
          _._merge)
)

,trip_id,route_id,first_departure,calitp_itp_id_x,calitp_url_number_x,have_rt,calitp_itp_id,calitp_url_number,date_x,start_time,_merge
0,882998,3493,17:46:00,300.0,0.0,left_only,NaN,NaN,NaN,NaN,left_only
1,881239,3481,09:00:00,300.0,0.0,left_only,NaN,NaN,NaN,NaN,left_only
2,881204,3481,17:42:00,300.0,0.0,left_only,NaN,NaN,NaN,NaN,left_only
3,880502,3479,10:32:00,300.0,0.0,left_only,NaN,NaN,NaN,NaN,left_only
4,880589,3479,20:03:00,300.0,0.0,left_only,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...
261,880533,3479,22:44:00,300.0,0.0,left_only,NaN,NaN,NaN,NaN,left_only
262,880583,3479,17:56:00,300.0,0.0,left_only,NaN,NaN,NaN,NaN,left_only
263,880584,3479,18:29:00,300.0,0.0,left_only,NaN,NaN,NaN,NaN,left_only
264,882000,3486,18:14:00,300.0,0.0,left_only,NaN,NaN,NaN,NaN,left_only


In [140]:
test_sched.sample()

,feed_key,trip_key,trip_id,route_id,calitp_itp_id_x,calitp_url_number_x,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours,calitp_itp_id_y,calitp_url_number_y,date_x,vehicle_id,have_rt,first_departure,calitp_itp_id,calitp_url_number,date_y,start_time,_merge
14,4.100255e+18,6.585285e+18,880481,3479,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,42.0,42.0,53340.0,56460.0,0.866667,NaN,NaN,NaN,NaN,left_only,14:49:00,NaN,NaN,NaN,NaN,left_only


In [136]:
test_sched>>count(_.route_id)>>arrange(_.n)

,route_id,n
10,3498,2
8,3493,10
9,3494,11
5,3487,12
7,3490,19
4,3486,22
1,3480,23
6,3489,25
3,3485,42
2,3481,48


In [141]:
alt.Chart((test_sched>>count(_.route_id))).mark_bar().encode(
    x='route_id',
    y='n'
)

alt.Chart(...)

In [144]:
test_sched>>select(_.trip_id,
                  _.first_departure)>>filter(_.first_departure.notnull())

,trip_id,first_departure
0,882998,17:46:00
1,881239,09:00:00
2,881204,17:42:00
3,880502,10:32:00
4,880589,20:03:00
...,...,...
261,880533,22:44:00
262,880583,17:56:00
263,880584,18:29:00
264,882000,18:14:00


In [147]:
# alt.Chart((test_sched)).mark_circle().encode(
#     x='route_id',
#     y='first_departure:T'
# )